In [1]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from urllib import request,response

# ***********************************PLEASE READ***********************************
#### This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves the "None" Value problem

## IMPORTANT: 
### 1) Please dont close the tab or the project will break, you can close the tab when you stop running the code.
### 2) If you wish, you can rerun the code and it will go scrap the website again and update the dataframe incase new games have been added since the creation of this project.

--------------------------------------------------------------------------
<h3> 1. Web scrapping to collect the Data

In [51]:
"""
*****************************************************INSTRUCTIONS*****************************************************
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, you can close the tab when the console displays 
"Scrapping Complete.".
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&sort=user_rating,desc'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")
#creates pandas dataframe 
def data_create():
    data = {
        'title': [],
        'release_date': [],
        'maturity_rating': [],
        'genre': [],
        'rating': [],
        'directors': [],
        'stars': [],
        'votes': []
    }
    global df
    df = pd.DataFrame(data= data)
data_create()


#func used to extract data by inputting the soup, the thing you want to find => "selector", and class => "class_selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
        return None

#continues looping till its done with finding a "next" link
try: 
    while True:
        #a big section that contains all the data of the videogame title
        game_info = soup.find_all("div", class_="lister-item-content")

        for i in game_info:
            #gets game name
            try:
                game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
            except AttributeError:
                game_name = None

            #gets release date
            try:
                game_release_date = extract_data(i, "span", "lister-item-year text-muted unbold")
                game_release_date = game_release_date.replace("(", "")
                game_release_date = game_release_date.replace(")", "")
                game_release_date = game_release_date.replace("I", "")
                game_release_date = int(game_release_date.replace("Video Game", "").strip())
            except:
                game_release_date = 0
            
            #gets maturity rating
            game_maturity = extract_data(i, "span", "certificate")

            #gets genre and split it into an array
            try:
                game_genre = extract_data(i, "span", "genre")
                if game_genre.find(",")!=-1:
                    game_genre = np.array(list(game_genre.split(",")))
                else:
                    game_genre = np.array(game_genre)
            except AttributeError:
                game_genre = None

            #get rating
            try:
                game_rating = float(i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip())
            except AttributeError:
                game_rating = 0

            #get Director & Stars and will split them, and will put "None" in values that dont exist
            #get both
            game_director_and_stars = extract_data(i, "p", "")
            game_director = None
            game_stars = None
            #if both director and star data exist for the title
            if game_director_and_stars.find("|")!=-1:
                game_director_full_text, separator, game_stars_full_text = game_director_and_stars.partition("|")

                game_director_full_text = game_director_full_text.strip()
                game_director_full_text = ''.join(game_director_full_text.splitlines())
                game_stars_full_text = game_stars_full_text.strip()
                game_stars_full_text = ''.join(game_stars_full_text.splitlines())

                director_text, separator, game_director = game_director_full_text.partition(":")
                star_text, separator, game_stars = game_stars_full_text.partition(":")

                #finds if there are many Directors/Stars or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))

            #didnt both Director and Star together, will try to find if only one of them exists        
            #if found Director
            elif game_director_and_stars.find("Director")!=-1:
                game_director_and_stars = ''.join(game_director_and_stars.splitlines())
                director_text, separator, game_director = game_director_and_stars.partition(":")
                game_director = game_director.replace("\n", "")
                #finds if there are many Directors or just one
                #there are multiple Directors
                if game_director.find(",")!=1:
                    game_director = np.array(list(game_director.split(",")))
                #there is only 1 Director
                else:
                    game_director = np.array(list(game_director))
                game_stars = None

            #if found Star
            elif game_director_and_stars.find("Star")!=-1:
                star_text, separator, game_stars = game_director_and_stars.partition(":")
                game_stars = game_stars.replace("\n", "")
                #finds if there are many Stars or just one
                #there are multiple Stars
                if game_stars.find(",")!=1:
                    game_stars = np.array(list(game_stars.split(",")))
                #there is only 1 Star
                else:
                    game_stars = np.array(list(game_stars))
                game_director = None
            else:
                game_director = None
                game_stars = None


            #get votes
            try: 
                game_votes = extract_data(i, "p", "sort-num_votes-visible")
                game_votes = ''.join(game_votes.splitlines())
                deleted_text, separator, game_votes = game_votes.partition(":")
                game_votes = game_votes.replace(",", "").strip()
                game_votes = int(game_votes)
            except:
                game_votes = 0

            #adds game details to dataframe
            df.loc[len(df.index)] = [game_name, game_release_date, game_maturity, game_genre, game_rating, game_director, game_stars, game_votes]
    #         display(df)


        #find button to click on next page 
        uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
        completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url

        print(completed_next_page_url)

        #replaces old url to completed_next_page_url
        url = completed_next_page_url
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()

        soup = BeautifulSoup(content, "html.parser")

    driver.quit()
except Exception as e:
    print(e)
    print("Scrapping Complete.")

D:\Anaconda\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=51&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=101&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=151&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=201&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=251&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=301&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=351&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=401&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating,desc&start=451&ref_=adv_nxt
https://www.imdb.com//search/title/?title_type=video_game&sort=user_rating

KeyboardInterrupt: 

In [98]:
df

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,['Action' ' Adventure' ' Drama'],9.8,['Matthew Gallant' ' Bruce Straley'],['Ashley Johnson' ' Troy Baker' ' Hana Hayes' ...,3525
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
2,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
3,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
4,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
...,...,...,...,...,...,...,...,...
33519,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33520,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33521,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33522,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0


In [6]:
#Save CSV
df.to_csv('ToCSV.csv', index=False)

<b>---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------<b>

<h3> 2. Analysis and Data Cleaning

In [52]:
df = pd.read_csv("ToCSV.csv")
df

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
1,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
2,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
3,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
4,The Witcher 3: Wild Hunt,2015,M,['Action' ' Adventure' ' Drama'],9.7,['Konrad Tomaszkiewicz'],['Doug Cockle' ' Denise Gough' ' Jo Wyatt' ' J...,29120
...,...,...,...,...,...,...,...,...
33518,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33519,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33520,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33521,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0


In [53]:
df.columns

Index(['title', 'release_date', 'maturity_rating', 'genre', 'rating',
       'directors', 'stars', 'votes'],
      dtype='object')

In [101]:
df.dtypes

title               object
release_date         int64
maturity_rating     object
genre               object
rating             float64
directors           object
stars               object
votes                int64
dtype: object

<b> Looking for Wrong Data<br>
    a. remove untitled upcoming games

In [35]:
for i in df["title"]:
    get_letter_index = i.find("Untitled")
    if get_letter_index != -1:
        untitled_game_to_be_deleted = i
        
        #gets index of games with "untitled" in them
        get_title_index = df.index[df['title']== untitled_game_to_be_deleted].tolist()
        print(get_title_index, i)
        
        #if the game is not called Untitled Goose Game, delete it (we wont delete this one because its the name of an actual game)
        if i != "Untitled Goose Game":
            df = df.drop(get_title_index)
            print("removed")
        else:
            print("The goose lives forever.")
print("Done")

[4639] Untitled Goose Game
The goose lives forever.
[14542] Untitled Iron Man game
removed
[15205] Untitled Indiana Jones Video Game
removed
[15614] Untitled Yakuza: Like A Dragon Sequel
removed
[17646] Untitled Haven PlayStation Game
removed
[19431] Untitled Pokèmon Videogame
removed
[19818] Untitled Tintin game
removed
[20618] Untitled Kaikan Phrase Game App
removed
[20811] Untitled Microsoft IP
removed
[23040] Untitled Mistwalker-Silicion Studio Game
removed
[24352] Untitled Hangman Game
removed
[24646] Untitled the X Collective Project
removed
[25042] Untitled Gaint Camel Game
removed
[28258] Untitled Clam Game
removed
[31458] Untitled Gun Game
removed
Done


In [36]:
#checks that the games are indeed deleted (only Untitled Goose Game will be there)
for i in df["title"]:
    title_index = i.find("Untitled")
    if title_index != -1:
        untitled_game_to_be_deleted = i
        print(untitled_game_to_be_deleted)
        

Untitled Goose Game


<b> b. replace release_date that is 0 with the actual date from google

In [48]:
df["release_date"].unique()

array([2016, 2013, 2022, 2018, 2015, 2021, 1998, 2023, 2010, 2019, 2004,
       2003, 1997, 1995, 2005, 2011, 2009, 2017, 2001, 2008, 2020, 2000,
       1999, 2002, 2014, 1991, 2007, 1990, 2012, 1988, 1994, 1996, 2006,
       1993, 1992, 1985, 1980, 1989, 1987, 1986, 1984, 1969, 1974, 1982,
       1983, 1975, 1981, 1971, 1972,    0, 1978, 1973, 1948, 1968, 1977,
       1979, 1976, 1952, 1958, 1962, 1970, 2024, 2027, 1950, 2030, 1964,
       1967, 2025, 2026], dtype=int64)

In [54]:
#gets index of wrong date data
wrong_date_index = np.array(list(df.index[df["release_date"] == 0]))
#gets title of wrong date data
wrong_date_title = np.array(list(df["title"].loc[df["release_date"] == 0]))

#puts index besides the name of game
wrong_data_combined = np.stack((wrong_date_index, wrong_date_title), axis=1)


print(wrong_data_combined)
# wrong_date_array = np.array(list((df["title"].loc[df["release_date"] == 0])))
# wrong_date_array[0]

[['4169' 'Total War: Three Kingdoms']
 ['5151' 'Deliver Us the Moon']
 ['5638' 'Super Street Fighter IV: 3D Edition']
 ...
 ['33433' 'Ion']
 ['33451' 'Never Say Die']
 ['33512' "Devil's Liminal"]]


In [50]:
print(wrong_data_combined[0])
print(wrong_data_combined[0][0])
print(wrong_data_combined[0][1])

['4169' 'Total War: Three Kingdoms']
4169
Total War: Three Kingdoms


<b> Actual Cell that cleans the release_date data

In [56]:
#WORKING
for i in range(len(df["title"].loc[df["release_date"] == 0])):
    #if successful to find date
    try:
        url = r"https://www.google.com/"
        game_name = wrong_data_combined[i][1]
        searchterm = f"video game {game_name} release date" # edit me

        driver = webdriver.Chrome()
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content,"html.parser")

        #changes language to english if needed
        english_link = soup.find("div", id="SIvCob").a.get("href")

        url = english_link
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, "html.parser")

        #seaches for search input box
        searchbox = driver.find_element(By.CLASS_NAME, "gLFyf")
        searchbox.send_keys(searchterm)
        searchbox.send_keys(Keys.RETURN)

        #Changes url to url of page after searching game release date
        url = driver.current_url
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, "html.parser")
        
        #get year
        #if full date is available as Big Header, need to separate year from the rest
        try:
            game_date = soup.find("div", class_="Z0LcW").text.strip()
            month, separator, year = game_date.partition(",")
            year = int(year)
            df["release_date"][wrong_data_combined[i][0]] = year

            print(url)
            print(f"Year: {year}")
            print(f"Index: {wrong_data_combined[i][0]}")
        
        #if only year is available as Big Header
        except:
            game_date = soup.find("div", class_="Z0LcW").text.strip()
            year = int(game_date)
            df["release_date"][wrong_data_combined[i][0]] = year

            print(url)
            print(f"Year: {year}")
            print(f"Index: {wrong_data_combined[i][0]}")
            
        driver.quit()        
        
    #didnt find date, will set it to 0
    except Exception as e:
        print("didnt find date")
        
        print(url)
        print(f"Index: {wrong_data_combined[i][0]}")
        
        print(e)
        #set unfound date to 0
        df["release_date"][wrong_data_combined[i][0]] = 0
        driver.quit()
        
    print("--------------------------------------------------------------------------------------------")


C:\Users\Ahmed\AppData\Local\Temp/ipykernel_10188/2662000769.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["release_date"][wrong_data_combined[i][0]] = year


https://www.google.com/search?q=video+game+Total+War%3A+Three+Kingdoms+release+date&source=hp&ei=RBkSZPLiMZfKkdUP8smLqAE&iflsig=AK50M_UAAAAAZBInVECWR6QAPdSr7gWAtlwUkoclMThM&ved=0ahUKEwjylMnQ0t79AhUXZaQEHfLkAhUQ4dUDCAg&uact=5&oq=video+game+Total+War%3A+Three+Kingdoms+release+date&gs_lcp=Cgdnd3Mtd2l6EANQAFidAWC_AWgAcAB4AIABAIgBAJIBAJgBAKABAQ&sclient=gws-wiz
Year: 2019
Index: 4169
--------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
#TESTING
for i in range(len(df["title"].loc[df["release_date"] == 0])):
    #if successful to find date
    try:
        #goes to google home page to search for first wrong release date
        url = r"https://www.google.com/"
        game_name = wrong_data_combined[i][1]
        searchterm = f"video game {game_name} release date" # edit me

        driver = webdriver.Chrome()
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content,"html.parser")

        #changes language to english if needed
        english_link = soup.find("div", id="SIvCob").a.get("href")

        url = english_link
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, "html.parser")

        #seaches for search input box
        searchbox = driver.find_element(By.CLASS_NAME, "gLFyf")
        searchbox.send_keys(searchterm)
        searchbox.send_keys(Keys.RETURN)

        #Changes url to url of page after searching game release date
        url = driver.current_url
        driver.get(url)
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, "html.parser")
        
        #get year
        #if full date is available as Big Header, need to separate year from the rest
        try:
            game_date = soup.find("div", class_="Z0LcW").text.strip()
            month, separator, year = game_date.partition(",")
            year = int(year)
            df["release_date"][wrong_data_combined[i][0]] = year

            print(url)
            print(f"Year: {year}")
            print(f"Index: {wrong_data_combined[i][0]}")
        
        #if only year is available as Big Header
        except:
            game_date = soup.find("div", class_="Z0LcW").text.strip()
            year = int(game_date)
            df["release_date"][wrong_data_combined[i][0]] = year

            print(url)
            print(f"Year: {year}")
            print(f"Index: {wrong_data_combined[i][0]}")
            
        driver.quit()        
        
    #didnt find date, will set it to 0
    except Exception as e:
        print("didnt find date")
        
        print(url)
        print(f"Index: {wrong_data_combined[i][0]}")
        
        print(e)
        #set unfound date to 0
        df["release_date"][wrong_data_combined[i][0]] = 0
        driver.quit()
        
    print("--------------------------------------------------------------------------------------------")


In [210]:
#RUN THIS AFTER DONE WITH TOP DATA CLEANING To Clean more data
url = r"https://www.google.com/"
game_name = "Cyberpunk 2077 - Phantom Liberty"
searchterm = f"video game {game_name} release date" # edit me

driver = webdriver.Chrome()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
soup = BeautifulSoup(content,"html.parser")

#changes language to english if needed
english_link = soup.find("div", id="SIvCob").a.get("href")

url = english_link
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
soup = BeautifulSoup(content, "html.parser")

#seaches for search input box
searchbox = driver.find_element(By.CLASS_NAME, "gLFyf")
searchbox.send_keys(searchterm)
searchbox.send_keys(Keys.RETURN)

#Changes url to url of page after searching game release date
url = driver.current_url
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
soup = BeautifulSoup(content, "html.parser")

#if there is alot of text that has the date mentioned in the middle
game_date = soup.find("span", class_="hgKElc").text.strip()
#finds all numbers in the game_date string 
unfiltered_years_list = [int(s) for s in re.findall(r'\b\d+\b', game_date)]
print(unfiltered_years_list)
#filters numbers between 1930 and 2030
for j in range(10):
    for i in unfiltered_years_list:
        if i < 1930 or i > 2030:
            unfiltered_years_list.remove(i)
year = max(unfiltered_years_list)
df["release_date"][wrong_data_combined[i][0]] = year

print(url)
print(f"Year: {year}")
print(f"Index: {wrong_data_combined[i][0]}")

[2077, 2077, 2023]
Year: 2023


In [184]:
a = "asfasfknsaf 2018 dsmkdsfkfkn 2077 kdskfknfdsnk August 21, 2001"
unfiltered_years_list = [int(s) for s in re.findall(r'\b\d+\b', a)]
unfiltered_years_list

[2018, 2077, 21, 2001]

In [185]:
for j in range(10):
    for i in unfiltered_years_list:
        if i < 1930 or i > 2030:
            unfiltered_years_list.remove(i)
    
print(unfiltered_years_list)

[2018, 2001]


In [192]:
filtered_year = max(unfiltered_years_list)
print(filtered_year)
print(type(filtered_year))

2018
<class 'int'>


In [125]:
df.iloc[4170]

title                                      Total War: Three Kingdoms
release_date                                                       0
maturity_rating                                                  NaN
genre                                                         Action
rating                                                           7.6
directors                                           ['Gaspar Janos']
stars              ['Anna Leong Brophy' ' Arthur Lee' ' Brendan H...
votes                                                            113
Name: 4170, dtype: object

In [119]:
df

,title,release_date,maturity_rating,genre,rating,directors,stars,votes
0,The Last of Us: Part I,2022,M,['Action' ' Adventure' ' Drama'],9.8,['Matthew Gallant' ' Bruce Straley'],['Ashley Johnson' ' Troy Baker' ' Hana Hayes' ...,3525
1,The Witcher 3: Wild Hunt - Blood and Wine,2016,M,['Action' ' Adventure' ' Drama'],9.8,['Konrad Tomaszkiewicz'],['Doug Cockle' ' MyAnna Buring' ' Antonia Bern...,8795
2,The Last of Us,2013,M,['Action' ' Adventure' ' Drama'],9.7,['Neil Druckmann' ' Bruce Straley'],['Troy Baker' ' Ashley Johnson' ' Hana Hayes' ...,69017
3,God of War: Ragnarök,2022,M,['Action' ' Adventure' ' Drama'],9.7,['Eric Williams'],['Christopher Judge' ' Sunny Suljic' ' Daniell...,11569
4,Red Dead Redemption II,2018,M,['Action' ' Adventure' ' Crime'],9.7,NaN,['Benjamin Byron Davis' ' Roger Clark' ' Rob W...,42853
...,...,...,...,...,...,...,...,...
33519,Through the Winter,2023,NaN,NaN,0.0,['Michael Smith'],['Rita Amparita' ' Hannah Beard' ' Xander M Gr...,0
33520,Atari 50: The Anniversary Celebration,2022,NaN,['Documentary' ' Action' ' Adventure'],0.0,NaN,['Al Alcorn' ' Nolan Bushnell' ' David Crane'],0
33521,Yars Revenge Enhanced,2022,NaN,['Action' ' Fantasy' ' Sci-Fi'],0.0,NaN,NaN,0
33522,Our Summers,2022,NaN,Romance,0.0,['Wavi'],['Reece Bridger'],0
